In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


url = "https://raw.githubusercontent.com/CharlesCLuo/Application-of-AI-in-Supply-Chain-Risk-Management-Series/main/Demand_Forecsting/demand_data.csv"
data = pd.read_csv(url, parse_dates=['Date'], index_col='Date')




scaler = MinMaxScaler()
data_normalized = data.copy()
data_normalized[['Demand']] = scaler.fit_transform(data[['Demand']])




train_size = int(len(data_normalized) * 0.8)
train_normalized, test_normalized = data_normalized.iloc[:train_size], data_normalized.iloc[train_size:]




X_train_normalized, y_train_normalized = train_normalized[['Demand']].values, train_normalized['Demand'].values
X_test_normalized, y_test_normalized = test_normalized[['Demand']].values, test_normalized['Demand'].values
X_train_normalized = X_train_normalized.reshape((X_train_normalized.shape[0], 1, 1))
X_test_normalized = X_test_normalized.reshape((X_test_normalized.shape[0], 1, 1))




rnn_model = Sequential()
rnn_model.add(SimpleRNN(50, activation='relu', input_shape=(1, 1), return_sequences=False))
rnn_model.add(Dropout(0.2))
rnn_model.add(Dense(1))




rnn_model.compile(optimizer='adam', loss='mean_squared_error')




rnn_history = rnn_model.fit(X_train_normalized, y_train_normalized, epochs=50, batch_size=32, validation_data=(X_test_normalized, y_test_normalized), verbose=1)




rnn_predictions_normalized = rnn_model.predict(X_test_normalized)




rnn_predictions_rescaled = scaler.inverse_transform(rnn_predictions_normalized.reshape(-1, 1))
y_test_rescaled = scaler.inverse_transform(y_test_normalized.reshape(-1, 1))




plt.figure(figsize=(14, 7))
plt.plot(data.index[train_size:], y_test_rescaled, label='Actual Demand (Test Data)', color='orange', linestyle='--')
plt.plot(data.index[train_size:], rnn_predictions_rescaled, label='RNN Forecasted Demand', color='purple', linestyle='-')
plt.axvline(x=data.index[train_size], color='black', linestyle=':', linewidth=1.5, label='Train-Test Split')
plt.title('Figure 3.2 RNN Model - Actual vs. Forecasted Demand')
plt.xlabel('Date')
plt.ylabel('Demand')
plt.legend()
plt.grid(True)
plt.show()




mae_original = mean_absolute_error(y_test_rescaled, rnn_predictions_rescaled)
rmse_original = np.sqrt(mean_squared_error(y_test_rescaled, rnn_predictions_rescaled))
mape_original = mean_absolute_percentage_error(y_test_rescaled, rnn_predictions_rescaled) * 100


print(f"RNN Model - MAE (original scale): {mae_original:.2f}")
print(f"RNN Model - RMSE (original scale): {rmse_original:.2f}")
print(f"RNN Model - MAPE (original scale): {mape_original:.2f}%")
